In [1]:
import torch
import pandas as pd
import numpy as np

from judgenet.modules.preprocess import SentenceEncoder

/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/JudgeNet/.env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
folder_path = "../data/mit_interview"

scores = pd.read_csv(f"{folder_path}/turker_scores_full_interview.csv")
transcripts = pd.read_csv(f"{folder_path}/interview_transcripts_by_turkers.csv")
prosody_features = pd.read_csv(f"{folder_path}/prosodic_features.csv")

print(scores.shape)
print(transcripts.shape)
print(prosody_features.shape)

(1378, 21)
(138, 2)
(690, 60)


# Extract aggregated scores

In [31]:
# Reduce scores to only Overall score aggregated across turkers
scores = scores[scores["Worker"]=="AGGR"][["Participant", "Overall"]]
print(scores)

     Participant   Overall
9             p1  5.297316
19            p3  4.414892
29            p4  4.494494
39            p5  5.457670
49            p6  5.106512
...          ...       ...
1338        pp83  6.045748
1348        pp84  5.710073
1358        pp85  5.626074
1368        pp86  4.853881
1377        pp89  4.960084

[138 rows x 2 columns]


In [32]:
scores_tensor = torch.tensor(scores["Overall"].values)
scores_tensor.shape

torch.Size([138])

# Extract and pool prosody features


In [33]:

# Loop through prosody features, taking the mean of each participants' 5 questions
# pooled_example = np.mean(prosody_features.iloc[0:5])

prosodic_features_pooled = pd.DataFrame()


for i in range(0,len(prosody_features),5):
    # Check first and last id to make sure chunk is all from same participant
    participant = prosody_features.iloc[i]["participant&question"].split('Q')[0]
    if participant != prosody_features.iloc[4+i]["participant&question"].split('Q')[0]:
        print("Misaligned chunk: ", i)
        break

    chunk = prosody_features.iloc[i:i+5, prosody_features.columns != "participant&question"]

    pooled = np.mean(chunk)
    # Add participant to row
    # pooled = pd.concat([pd.Series([participant.lower()], index=["Participant"]), pooled])

    prosodic_features_pooled = pd.concat([prosodic_features_pooled, pooled], axis=1)

prosodic_features_pooled = prosodic_features_pooled.transpose()

/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/JudgeNet/.env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/JudgeNet/.env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Users/jakeisrael/USC/Spring 2023/Multimodan Human Communication/project/JudgeNet/.env/lib/python3.8/site-packages/numpy/core

In [34]:
print(prosodic_features_pooled)
prosodic_features_pooled.shape

     duration    energy     power  min_pitch   max_pitch  mean_pitch  \
0   34.186796  0.011510  0.000333  78.064294  340.516454  129.307639   
0   89.165042  0.030218  0.000332  70.536185  400.408060  128.747442   
0   47.795525  0.006202  0.000122  74.847180  359.197092  137.312149   
0   30.842617  0.002352  0.000071  79.266129  311.060876  122.560903   
0   81.745013  0.003205  0.000040  73.461743  355.214130  107.848640   
..        ...       ...       ...        ...         ...         ...   
0   28.907788  0.003143  0.000108  75.106137  375.449500  240.799891   
0   52.262092  0.008353  0.000146  77.471544  393.760530  209.208918   
0   62.115933  0.007095  0.000112  71.071074  367.137037  108.901779   
0   76.643092  0.010589  0.000132  78.332326  353.811432  128.650234   
0   80.570025  0.019874  0.000245  72.692857  367.095029  141.030662   

     pitch_sd   pitch_abs  pitch_quant  pitchUvsVRatio  ...  avgDurPause  \
0   25.242104  176.176383   121.943026        0.951904  ...

(138, 57)

In [35]:
prosody_tensor = torch.tensor(prosodic_features_pooled.values)
prosody_tensor.shape

torch.Size([138, 57])

# Prune Transcript down to just interviewee speech and extract BERT features

Note: Kernel was crashing when extracting all the rows, so I'm batching it

In [15]:
def extract_bert(transcript):
    split = transcript.split('Interviewee:')

    # Trim transcript to Interviewee speech
    cleaned = ""
    for row in split[1:]:
        cleaned += row.split('|')[0]

    tokens = se.tokenize(cleaned)
    return se.encode_batched_tokens(tokens)

se = SentenceEncoder()
current_pos = 120

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
lexical_features = []

for i in range(current_pos, current_pos+20):
    if i < len(transcripts):
        transcript = transcripts.iloc[i]["Transcript"]
        embedding = extract_bert(transcript)
        lexical_features.append(embedding)

current_pos = current_pos + 20
print(f"finished up to row row {i}")
stacked = torch.stack(lexical_features)
torch.save(stacked, f"{folder_path}/features/lexical_batched/{i}.csv")

finished up to row row 139


In [18]:
feats = []
filenames = ["19.pt", "39.pt", "59.pt", "79.pt", "99.pt", "119.pt", "139.pt"]
for name in filenames:
    feats.append(torch.load(f"{folder_path}/features/lexical/{name}"))

In [27]:
lexical_tensor = torch.cat(feats, dim=0)

In [36]:
print(f"Shape of scores: {scores_tensor.shape}")
print(f"Shape of prosody: {prosody_tensor.shape}")
print(f"Shape of lexical: {lexical_tensor.shape}")

Shape of scores: torch.Size([138])
Shape of prosody: torch.Size([138, 57])
Shape of lexical: torch.Size([138, 768])


In [37]:
torch.save(scores_tensor, f"{folder_path}/features/scores.pt")
torch.save(prosody_tensor, f"{folder_path}/features/prosody.pt")
torch.save(lexical_tensor, f"{folder_path}/features/lexical.pt")